# Python Learn by Doing: ENSO, Your Turn! Option 1 Answer Key

**Developed By:** Dr. Kerrie Geil, Mississippi State University

**Original Development Date:** June 2024

**Package Requirements:** xarray, netcdf4, numpy, pandas, scipy, matplotlib, jupyter, geopandas

**Links:** **[OSF project link](https://osf.io/zhpd5/)**, [link to this notebook on github]()

---
**Assignment:**

Using a shapefile with country boundaries, show a table (use a pandas dataframe) of the percent area of each country in South America where there is statistically significant anomalous temperature and precipitation during strong El Nino and La Nina events. 

&emsp;Hints:
- Use World_Countries_Generalized.shp for country boundaries and subset to South America. Here's the code to download the shapefile
```
from urllib.request import urlretrieve

# create a folder for data downloads
if not os.path.exists('../data/World_Countries'):
    os.makedirs('../data/World_Countries')

# filenames to save data to and download urls
base_filename='../data/World_Countries/World_Countries'

shpfile_info=  {'.cpg':'https://osf.io/5xrgc/download',
                '.dbf':'https://osf.io/3a6rp/download',
                '.prj':'https://osf.io/43mnp/download',
                '.shp':'https://osf.io/r4dez/download',
                '.shp.xml':'https://osf.io/s4cvy/download',
                '.shx':'https://osf.io/kp6cm/download'}    
for ext,url in shpfile_info.items():
    filename=base_filename+ext
    print('downloading',filename)
    urlretrieve(url,filename) # download and save data
```
- Repeat the appropriate steps from science questions 1 and 3
- Use the country boundaries and your gridded results to quantify how much area of each country experiences statistically significant anomalous T and PR (use p<=0.1)
- Format your results in a pandas dataframe. At minimum, your final dataframe should have columns for COUNTRY, T_PERCENT_AREA, PR_PERCENT_AREA

# Import packages and define workspace

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import scipy.stats as ss
import geopandas as gpd
import warnings
import os
from urllib.request import urlretrieve
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf

import shapely
import xagg
# from cdo import *
# cdo   = Cdo()  

In [2]:
# create a folder for data downloads
if not os.path.exists('../data/World_Countries'):
    os.makedirs('../data/World_Countries')

In [ ]:
# filenames to save data to and download urls
base_filename='../data/World_Countries/World_Countries'

shpfile_info=  {'.cpg':'https://osf.io/5xrgc/download',
                '.dbf':'https://osf.io/3a6rp/download',
                '.prj':'https://osf.io/43mnp/download',
                '.shp':'https://osf.io/r4dez/download',
                '.shp.xml':'https://osf.io/s4cvy/download',
                '.shx':'https://osf.io/kp6cm/download'}

# files we've already downloaded
nino_f = '../data/nino34_anomalies_monthly_NOAA.txt'
t_f = '../data/tavg_monthly_BerkeleyEarth.nc'
pr_f = '../data/precip_monthly_PRECL_NOAA.nc'

# Obtaining the data

We'll use a shapefile of country boundaries called [World Countries, originally obtained from ESRI ArcGIS Hub](https://hub.arcgis.com/datasets/esri::world-countries/explore) in June 2024 and copied to the [enso component of the MSU_py_training OSF project](https://osf.io/e726y/). 

<br>
<font color="green"><b>
**NOTE: You only need to run the following urlretrieve cell once. The data files will then be located on your computer. Files total approximately 130MB in size.**
</b></font> 

In [ ]:
# download shapefile

for ext,url in shpfile_info.items():
    filename=base_filename+ext
    print('downloading',filename)
    urlretrieve(url,filename) # download and save data

# Data Cleaning

We'll copy over the data cleaning steps from the enso_analysis.ipynb as well as subset global country boundaries to South American countries 

In [ ]:
# data cleaning copied from enso_analysis.ipynb

year_start = '1948'
year_end = '2023'
base_start = '1981'
base_end = '2010'

dates=pd.date_range('1870-01-01','2024-12-01',freq='MS')

In [ ]:
# data cleaning copied from enso_analysis.ipynb

# Nino 3.4 data
nino_raw=pd.read_csv(nino_f,sep=r'\s+',skiprows=1,skipfooter=7,header=None,index_col=0,na_values=-99.99,engine='python')
nino=nino_raw.to_numpy().flatten() 
nino=xr.DataArray(nino,name='nino',dims='time',coords={'time':dates}) 
nino.attrs['standard_name']='nino3.4 index'
nino.attrs['units']='C'
nino=nino.sel(time=slice(year_start,year_end))

# precipitation data
ds=xr.open_dataset(pr_f)
pr=ds.precip
pr=pr.sel(time=slice(year_start,year_end)) 
pr=pr.reindex(lat=pr.lat[::-1]) 
pr.coords['lon']=xr.where(pr.coords['lon']>180,pr.coords['lon']-360,pr.coords['lon'])
roll_len=len(pr.lon)//2
pr=pr.roll(lon=roll_len,roll_coords=True)
pr_base=pr.sel(time=slice(base_start,base_end)) 
pr_clim=pr_base.groupby(pr_base.time.dt.month).mean('time') 
pr_anom=pr.groupby(pr.time.dt.month) - pr_clim  
pr_anom.attrs['standard_name']='pr anomaly'
pr_anom.attrs['units']='mm/day'

# temperature data
ds=xr.open_dataset(t_f)
dates=pd.date_range('1750-01-01','2024-03-01',freq='MS')
ds['time']=dates
ds=ds.rename({'month_number':'month','latitude':'lat','longitude':'lon'})
t_anom_5180=ds.temperature
clim_5180=ds.climatology
t=t_anom_5180.groupby(t_anom_5180.time.dt.month)+clim_5180
t_base=t.sel(time=slice(base_start,base_end))  # subset in time
clim_8110 = t_base.groupby(t_base.time.dt.month).mean('time')  # long term means for each month
t_anom=t.groupby(t.time.dt.month)-clim_8110
t_anom=t_anom.sel(time=slice(year_start,year_end))
t_anom=t_anom.rename('tavg')
t_anom.attrs['standard_name']='T anomaly'
t_anom.attrs['units']='C'

# check first and last time is the same for all data
variables=[nino, pr_anom, t_anom] # list of arrays
for var in variables:
    print(var.name, var.time[0].data,var.time[-1].data)

# clean up
del ds, nino_raw, pr, pr_base, pr_clim,  t, t_anom_5180, t_base

In [ ]:
# load shapefile of global country boundaries
countries=gpd.read_file(base_filename+'.shp')
countries

In [ ]:
countries.crs
# options:
# 1) project both countries and data to equal area projections
# 2) unproject countries and use the built in geopandas function to compute are on non-projected country polygons


In [ ]:
# trying unproject first

# subset to south america
countries=countries.loc[(countries['CONTINENT']=='South America')
                        &(countries['LAND_TYPE'].str.contains('Primary land'))].reset_index(drop=True)
countries=countries.to_crs("epsg:4326")
countries

# array(['Argentina', 'Aruba', 'Bolivia', 'Bonaire', 'Brazil', 'Chile',
#        'Colombia', 'Curacao', 'Ecuador', 'Falkland Islands',
#        'French Guiana', 'Guyana', 'Paraguay', 'Peru', 'Suriname',
#        'Trinidad and Tobago', 'Uruguay', 'Venezuela'], dtype=object)

In [ ]:
# # countries.crs.is_geographic
# # not countries.crs
# geod = countries.crs.get_geod()
# geod

In [ ]:
# countries.geometry[0].geoms[0].exterior

In [ ]:
# # shapely.geometry.polygon.orient(countries.geometry[0].geoms[0], 1)
# # countries.geometry.geom_type#.crs#[9].geom_type
# # all(countries.geometry.geom_type).str.contains(['MultiPolygon','Polygon'])
# countries.geometry.geom_type.isin(['MultiPolygon','Polygon'])
# all(countries.geometry.geom_type.isin(['MultiPolygon','Polygon']))

In [ ]:
# geod.geometry_area_perimeter(orient(geom, 1))[0]
# test= countries.geometry[0].geoms[0]
# test.exterior

# geod = countries.crs.get_geod()
# area_km2=[geod.geometry_area_perimeter(shapely.geometry.polygon.orient(geom,1))[0]/(1000^2) for geom in countries.geometry[0].geoms]
# area_km2


In [ ]:
from shapely.geometry.polygon import orient
def gpd_geographic_area(geodf):
    if not geodf.crs and geodf.crs.is_geographic:
        raise TypeError('geodataframe should have geographic coordinate system')
        
    geod = geodf.crs.get_geod()
    def area_calc(geom):
        if geom.geom_type not in ['MultiPolygon','Polygon']:
            return np.nan
        
        # For MultiPolygon do each separately
        if geom.geom_type=='MultiPolygon':
            return np.sum([area_calc(p) for p in geom.geoms])/(1000^2)

        # orient to ensure a counter-clockwise traversal. 
        # See https://pyproj4.github.io/pyproj/stable/api/geod.html
        # geometry_area_perimeter returns (area, perimeter)
        return geod.geometry_area_perimeter(orient(geom, 1))[0]/(1000^2)
    
    return geodf.geometry.apply(area_calc)

In [ ]:
countries['area_km2']=gpd_geographic_area(countries)
countries

In [ ]:
# # this function operates on a single polygon or multipolygon
# # and can be applied to a geodataframe
# def area_calc(geom):
#     if not all(geom.geom_type.isin(['MultiPolygon','Polygon'])):
#         raise TypeError('some geometries are not polygon or multipolygon')

#     geod = geom.crs.get_geod()        
#     if geom.geom_type=='MultiPolygon':
#         # For multipolygon do each polygon separately and sum
#         return np.sum([geod.geometry_area_perimeter(shapely.geometry.polygon.orient(p,1))[0]/(1000^2) for p in geom.geoms])
#     else:
#         return geod.geometry_area_perimeter(shapely.geometry.polygon.orient(geom,1))[0]/(1000^2)

In [ ]:
# # geod = countries.crs.get_geod()
# countries.geometry.apply(area_calc)
# # countries.apply(area_calc)

In [ ]:
# # get country areas
# geod = countries.crs.get_geod()




# from shapely.geometry.polygon import orient
# def gpd_geographic_area(geodf):
#     if not geodf.crs and geodf.crs.is_geographic:
#         raise TypeError('geodataframe should have geographic coordinate system')
        
#     geod = geodf.crs.get_geod()
#     def area_calc(geom):
#         if geom.geom_type not in ['MultiPolygon','Polygon']:
#             return np.nan
        
#         # For MultiPolygon do each separately
#         if geom.geom_type=='MultiPolygon':
#             return np.sum([area_calc(p) for p in geom.geoms])

#         # orient to ensure a counter-clockwise traversal. 
#         # See https://pyproj4.github.io/pyproj/stable/api/geod.html
#         # geometry_area_perimeter returns (area, perimeter)
#         return geod.geometry_area_perimeter(orient(geom, 1))[0]
    
#     return geodf.geometry.apply(area_calc)

In [ ]:
# # count how many different countries and see their names
# print(len(countries.COUNTRY.unique()))
# countries.COUNTRY.unique()

# Begin Main Analysis

First, we need to know when El Nino and La Nina events occurred so we'll copy over code from question 1 in enso_analysis.ipynb (How many strong El Nino and La Nina events have occurred from 1948 to 2023?). We only need the code that creates the array `nino_events` 

In [ ]:
# copied from question 1 in enso_analysis.ipynb

# constants based on our criteria
nmonths=5
event_thresh=0.6

# first calculate the 5-month rolling mean
nino_rollmean=nino.rolling(time=nmonths,center=True).mean()

# create an array to hold our results and initialize to nan
# this array is where we will fill values with +1,-1
nino_events=nino_rollmean.copy() 
nino_events[:]=np.nan

# now loop through months and fill +1, -1 for windows of 5 months that meet our criteria
for i,value in enumerate(nino_rollmean):
    # La Nina conditions
    if  value < -event_thresh:
        # possible La Nina conditions, look forward 4 more months
        window=nino_rollmean[i:i+nmonths]
        if all(window < -event_thresh):
            nino_events[i:i+nmonths] = -1

    # El Nino conditions
    if  value > event_thresh:
        # possible El Nino conditions, look forward 4 more months
        window=nino_rollmean[i:i+nmonths]
        if all(window > event_thresh):
            nino_events[i:i+nmonths]=1    

Next, we need to build our El Nino composite and determine statistical significance. We'll copy over the relevant code from question 3 in enso_analysis.ipynb

In [ ]:
# copied from question 3 in enso_analysis.ipynb

# starting with el nino conditions, temperature
# get temperature anomalies only for times during strong el nino conditions
t_nino=t_anom.where(nino_events==1,drop=True)

# now separate out winter DJF months
# this is sample 1: winter months during strong el nino conditions
t_nino_DJF=t_nino.groupby(t_nino.time.dt.season)['DJF'] 

# make a composite
t_nino_DJF_composite=t_nino_DJF.mean('time',keep_attrs=True)

# create a t sample that include all winter months DJF when there are not strong el nino conditions

# all months that don't fall in strong nino events
t_other=t_anom.where(nino_events!=1,drop=True) 

# pull out just DJF months
# this is sample 2: all winter months that are NOT during strong el nino conditions
t_other_DJF=t_other.groupby(t_other.time.dt.season)['DJF'] 

print('t nino and non-nino sample sizes:',t_nino_DJF.shape[0],t_other_DJF.shape[0]) 

# t-test for difference in means 
t_sigtest = ss.ttest_ind(t_nino_DJF, t_other_DJF, axis=0, equal_var=False)
# numpy --> xarray
t_nino_pval = xr.DataArray(t_sigtest.pvalue, coords={'lat':('lat',t_nino.coords['lat'].data),'lon':('lon',t_nino.coords['lon'].data)})


# plot el nino temperature anomalies where statistically significant at 90% level
pval=0.1

fig=plt.figure(figsize=(12,8))

ax=fig.add_subplot(111,projection=ccrs.PlateCarree())
ax.add_feature(cf.COASTLINE.with_scale("50m"),lw=0.3)
ax.add_feature(cf.BORDERS.with_scale("50m"),lw=0.3)
t_nino_DJF_composite.where(t_nino_pval<pval).plot(cmap='RdBu_r',cbar_kwargs={'shrink':0.9,'orientation':'horizontal','pad':0.05})
plt.title(f'winter mean temperature anomalies\n during strong El Nino conditions (p < {pval})')

plt.show()


Now we can use our shapefile to answer the question: which South American countries experience statistically significant anomalous temperature and precipitation over at least 50% of the country's area during strong El Nino? 

In [ ]:
t_nino_DJF_composite

In [ ]:
# first create a pandas dataframe from our results t_nino_DJF_composite and t_nino_pval

# convert each xarray data array object to a pandas data frame
t_nino_DJF_composite_df=t_nino_DJF_composite.to_dataframe().reset_index(level=[0,1])

t_nino_pval.name='pval'
t_nino_pval_df=t_nino_pval.to_dataframe().reset_index(level=[0,1])

# # merge pvalues into t_nino_DJF_composite_df
t_nino_DJF_composite_df=t_nino_DJF_composite_df.merge(t_nino_pval_df, how='left',on=['lat','lon'])
t_nino_DJF_composite_df

In [ ]:
t_nino_DJF_composite_df.crs

In [ ]:
# now use the xagg package to compute the overlaps between grid cell polygons and country polygons
# pixel_overlaps computes the relative area of overlap for each grid cell polygon
# it takes an xarray dataset and a geopandas dataframe as inputs
# see https://xagg.readthedocs.io/en/latest/xagg.html#xagg.wrappers.pixel_overlaps
# and https://xagg.readthedocs.io/en/latest/xagg.html#xagg.core.get_pixel_overlaps
weightmap = xagg.pixel_overlaps(t_nino_DJF_composite,countries,subset_bbox=False)

In [ ]:
# xa.pixel_overlaps function returns an object that contains 
# 1) a pandas dataframe with the grid cell polygon overlap information, 
# 2) a dictionary containing the xarray data array source grid info, 
# 3) a pandas series of geometry objects containing the geopandas source geometry info

# access the dataframe with .agg
# weightmap.agg.head()

xagg.pixel_overlaps calculates how much relative area of a country polygon that each grid cell that intersects the country occupies. The column rel_area contains this information

In [ ]:
# the relative area of each pixel intersecting the first polygon/mulipolygon (Argentina)
# would be indexed like this, which returns a pandas series of 351 values
# in this case 351 grid cells of our xarray data array intersect the Argentina polygon
# weightmap.agg['rel_area'][1][0]

In [ ]:
# the coordinates that correspond to the rel_area values (grid cell polygon centroids)
# would be indexed like this, which returns a list of 351 (lat,lon) tuples
# weightmap.agg['coords'][1]

In [ ]:
weightmap.agg#.crs

For each country we want to create a dataframe where each row contains data for a single grid cell: relative area, lat, lon values, data value, pvalue. We'll start with the first country Argentina

In [ ]:
# coordinates lat,lon to dataframe
df=pd.DataFrame(weightmap.agg['coords'][11],columns=['lat','lon'])

# add rel_area as a dataframe column
df['rel_area']=weightmap.agg['rel_area'][11][0].reset_index(drop=True)

# join the tavg and pval info
df=df.merge(t_nino_DJF_composite_df, how='left',on=['lat','lon'])
df

In [ ]:
# test

In [ ]:
# xagg.diag.diag_fig(weightmap,0,t_nino_DJF_composite)

# test=xagg.fix_ds(t_nino_DJF_composite)
# weightmap.diag_fig(0,test)

# grid_polygon_info = xagg.subset_find(t_nino_DJF_composite,weightmap.source_grid)
# weightmap.diag_fig(1,grid_polygon_info)
# grid_polygon_info

weightmap.diag_fig(11,t_nino_DJF_composite.to_dataset())

In [ ]:
# data_subset=t_nino_DJF_composite.sel(lat=slice(-20,0),lon=slice(-82,-68))
data_subset=t_nino_DJF_composite.sel(lat=slice(9.5,11.5),lon=slice(-62,-60))
# data_subset
fig=plt.figure(figsize=(6,6))

ax=fig.add_subplot(111,projection=ccrs.PlateCarree())
ax.add_feature(cf.COASTLINE.with_scale("50m"),lw=0.3)
ax.add_feature(cf.BORDERS.with_scale("50m"),lw=0.3)

# Make a copy
cmap = plt.colormaps.get_cmap("RdBu_r").copy()

# Choose the color
cmap.set_bad('grey')

data_subset.plot(cmap=cmap)#,cbar_kwargs={'shrink':0.9,'orientation':'horizontal','pad':0.05})

# Gridlines
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=['x','y','bottom','left'],
                    linewidth=1, color='dimgray', alpha=0.5, linestyle=':')

In [ ]:
# to get the relative area of the country where nino anomalies meet the 90% confidence level
area_impacted=df.loc[df['pval']<=0.1].rel_area.sum()
area_impacted

To get the result for all countries we can write a function and loop. 

In [ ]:
def percent_area_impacted(coords,rel_area,xr_pval):
    df=pd.DataFrame(coords,columns=['lat','lon'])
    df['rel_area']=rel_area.reset_index(drop=True)
    
    xr_pval.name='pval'
    pval_df=xr_pval.to_dataframe().reset_index(level=[0,1])
    df=df.merge(pval_df, how='left',on=['lat','lon'])

    area_impacted=df.loc[df['pval']<=0.1].rel_area.sum()*100.
    area_impacted = round(area_impacted)
    return area_impacted

In [ ]:
results={}
for index,row in weightmap.agg.iterrows():
    answer=percent_area_impacted(row.coords,row.rel_area[0],t_nino_pval)
    results[row.COUNTRY]=answer
results

## Side note about calculating irregular polygon areas from gridded/raster data

Here, our gridded data is nan over ocean grids (or grids that are mostly ocean). This means a grid cell with a value of nan could slighly overlap where land is present. We have to take that into consideration when interpreting the accuracy/precision of these results.

Take, for example, Trinidad and Tobago. The area we calculated is 98% instead of 100% because of the way the grid cells align with the coastline. This will be the case for any country with a coast.

Let's take a closer look.

In [ ]:
# using xagg's diagnostic figure to look at which grid cells 
# overlap Trinidad and Tobago (row 11 of countries dataframe)
weightmap.diag_fig(11,t_nino_DJF_composite.to_dataset())

In [ ]:
# making a plot to look at which grid cells of our data are considered ocean (nan)

fig=plt.figure(figsize=(6,6))
ax=fig.add_subplot(111,projection=ccrs.PlateCarree())
ax.add_feature(cf.COASTLINE.with_scale("50m"),lw=0.3)
# color nan grey
cmap = plt.colormaps.get_cmap("RdBu_r").copy()
cmap.set_bad('grey') 

# subset based on the above plot's lat and lon
t_nino_DJF_composite.sel(lat=slice(9.5,11.5),lon=slice(-62,-60)).plot(cmap=cmap)

In [ ]:
# access the dictionary with .source_grid
weightmap.source_grid

In [ ]:
weightmap.geometry

In [ ]:
pr_test=pr_anom.isel(time=0).to_dataset()
pr_test

In [ ]:
# testdf=pr_test.stack(cell=('lat','lon')).to_dataframe()#.to_pandas()#.drop_vars(['lat','lon'])
testdf=pr_test.to_dataframe().reset_index(level=[0,1])#.to_pandas()#.drop_vars(['lat','lon'])

# testdf=testdf.reindex()
testdf

In [ ]:
weightmap = xa.pixel_overlaps(pr_test,countries,subset_bbox=False)
weightmap

In [ ]:
rel_area=weightmap.agg['rel_area']
rel_area

In [ ]:
coords=weightmap.agg['coords']
coords

In [ ]:
# the relative area of each pixel intersecting the first polygon/mulipolygon
# would be indexed like this, which returns a pandas series of 351 values
rel_area[0][0]

In [ ]:
# the coordinates that correspond to the rel_area valuse/pixels
# would be indexed like this, which returns a list of 351 (lat,lon) tuples
coords[0]

In [ ]:
# for each polygon, we can create a df of pixel rel_area, center coordinates, and data value like this
df=pd.DataFrame(coords[0],columns=['lat','lon'])
df['rel_area']=rel_area[0][0]
df=df.merge(testdf, how='left',on=['lat','lon'])
df

In [ ]:
# to find the area where the data variable meets some criteria 
# for example, the magnitude of the pr anomaly gt 0.5

df_sub=df[abs(df['precip'])>=0.5]
relarea_criteria=df_sub['rel_area'].sum()
relarea_criteria

# now multiply by the area of the polygon

In [ ]:
# peru=countries.loc[9]
peru=countries[countries.COUNTRY=='Peru']
# peru
weightmap2 = xa.pixel_overlaps(pr_anom_ds,peru,subset_bbox=False)
weightmap2.agg['rel_area']

In [ ]:
weightmap2.agg

In [ ]:
weightmap2.agg.rel_area[0][0]

In [ ]:
sum(weightmap2.agg.rel_area[0][0])

In [ ]:
# np.array(weightmap2.agg.coords[0]).shape#.rel_area[0][0]

weightmap2.agg.coords[0]#[1][1])

In [ ]:
# # Querying polygon by column of the polygon `gdf`
# weightmap.diag_fig({'name':'Alaska'},ds)

# Plotting the first polygon in the polygon `gdf`
weightmap.diag_fig(0,pr_anom_ds)

In [ ]:
import rioxarray as rio
pr_anom_ds=pr_anom.to_dataset()
# pr_anom_ds=pr_anom_ds.rename({'lat':'y','lon':'x'})
# pr_anom_ds=pr_anom_ds.rio.write_crs('epsg:4326')
# pr_test=pr_anom_ds.rio.reproject('epsg:3857')
pr_test=pr_anom_ds
pr_test

In [ ]:
# pr_test=pr_test.rename({'y':'lat','x':'lon'})
pr_test=xa.auxfuncs.get_bnds(pr_test)
pr_test

In [ ]:
grid_poly=xa.core.create_raster_polygons(pr_test)
# grid_poly,grid_dict=xa.core.create_raster_polygons(pr_test) # doesn't work this way

gdf_pixels=grid_poly['gdf_pixels']
gdf_pixels

In [ ]:
gdf_pixels.crs

In [ ]:
weights = np.cos(np.deg2rad(pr_anom.lat))
weights.name = "weights"
# weights

In [ ]:
pr_weighted = pr_anom.weighted(weights)
pr_weighted

In [ ]:
# cdo gridarea yourdata.nc gridarea.nc 
ds=xr.open_dataset(pr_f)
ds